In [1]:
import pandas as pd
import numpy as np
import math
import time
import collections
import xlsxwriter

In [33]:
pd_data = pd.read_excel('91_loan_1_from_json.xlsx')

# 毫秒级时间戳转换
pd_data['getTime'] = pd_data['getTime'].map(lambda x:time.mktime(time.strptime(str(x),'%Y%m%d%H')))
pd_data['contractDate'] = pd_data[['getTime','contractDate']].apply(lambda x:x[0]-x[1]//1000,axis=1)

In [34]:
pd_data.replace(['non','none','None','NONE',
                       'null','NULL','Null','[]','[ ]','{}','{ }',
                        ' ', '', '()', '( )'], np.nan, inplace=True) 


# 申请金额取中值,string转换为int型
pd_data['borrowAmount'] = pd.to_numeric(pd_data['borrowAmount'], errors='ignore')
d={-7:500, -6:1500, -5:2500, -4:3500, -3:5000, -2:7000, -1:9000, 0:np.nan, 1:15000}

pd_data['borrowAmount'] = pd_data['borrowAmount'].map(lambda x:d.get(x) if x<=1 else 20000*x-10000)


# 转换为int型
pd_data['loanPeriod'] = pd.to_numeric(pd_data['loanPeriod'], errors='ignore')
pd_data['repayState'] = pd.to_numeric(pd_data['repayState'], errors='ignore')
pd_data['borrowType'] = pd.to_numeric(pd_data['borrowType'], errors='ignore')
pd_data['arrearsAmount'] = pd.to_numeric(pd_data['arrearsAmount'], errors='ignore').map(lambda x:x/100000.0)
pd_data['certNo'] = pd.to_numeric(pd_data['certNo'], errors='ignore')

period = (('10d',60*60*24*10),
          ('30d',60*60*24*30),
          ('90d',60*60*24*90),
          ('180d',60*60*24*180),
          ('360d',60*60*24*360),
          ('all',2**31 - 1))

id_data = pd.DataFrame(index=pd_data['certNo'].drop_duplicates().sort_values().values)

In [35]:
for prdname, prdtime in period:
    prddata = pd_data[pd_data['contractDate']<prdtime]

#### PART 1 申请状态   

    #### n天内申请总次数
    id_data['jyzx_n_Inquiry_' + prdname] = prddata[['certNo','contractDate']].groupby('certNo').count()
    
    #### n天内申请总次数（申请状态=未知）
    id_data['jyzx_n_Inquiry_Unknown_' + prdname] = prddata[prddata['borrowState'].isin([0])][['certNo','borrowState']].groupby('certNo').count()
    
    #### n天内申请总次数（申请状态=拒贷）
    id_data['jyzx_n_Inquiry_Rejected_' + prdname] = prddata[prddata['borrowState'].isin([1])][['certNo','borrowState']].groupby('certNo').count()  
    
    #### n天内申请总次数（申请状态=批贷已放款）
    id_data['jyzx_n_Inquiry_Approved_' + prdname] = prddata[prddata['borrowState'].isin([2])][['certNo','borrowState']].groupby('certNo').count()

    #### n天内申请总次数（申请状态=待放款）
    id_data['jyzx_n_Inquiry_Awaiting_' + prdname] = prddata[prddata['borrowState'].isin([3,6])][['certNo','borrowState']].groupby('certNo').count()
    
    #### n天内申请总次数（申请状态=申请人放弃申请）
    id_data['jyzx_n_Inquiry_Aborted_' + prdname] = prddata[prddata['borrowState'].isin([4])][['certNo','borrowState']].groupby('certNo').count()  
    
    #### n天内申请总次数（申请状态=审核中）
    id_data['jyzx_n_Inquiry_Checking_' + prdname] = prddata[prddata['borrowState'].isin([5])][['certNo','borrowState']].groupby('certNo').count()   
    
    #### n天内申请总次数（申请状态=批贷已放款/待放款）
    id_data['jyzx_n_Inquiry_Pass_' + prdname] = prddata[prddata['borrowState'].isin([2,3,6])][['certNo','borrowState']].groupby('certNo').count()
    
    #### n天内申请次数通过率
    id_data['jyzx_n_Inquiry_Pass_rate_' + prdname] =\
    id_data[['jyzx_n_Inquiry_' + prdname,'jyzx_n_Inquiry_Pass_' + prdname]]\
    .apply(lambda x:round(x[1]/x[0],2) if x[0]>0 else np.nan, axis=1)
    
    

    
    #### n天内申请总金额    
    id_data['jyzx_sum_Inquiry_' + prdname] = prddata[['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内申请总金额（申请状态=未知）
    id_data['jyzx_sum_Inquiry_Unknown_' + prdname] = prddata[prddata['borrowState']==0][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n内申请总金额（申请状态=拒贷）
    id_data['jyzx_sum_Inquiry_Rejected_' + prdname] = prddata[prddata['borrowState']==1][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n内申请总金额（申请状态=批贷已放款）
    id_data['jyzx_sum_Inquiry_Approved_' + prdname] = prddata[prddata['borrowState']==2][['certNo','borrowAmount']].groupby('certNo').sum()

    #### n内申请总金额（申请状态=待放款）
    id_data['jyzx_sum_Inquiry_Awaiting_' + prdname] = prddata[prddata['borrowState'].isin([3,6])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n内申请总金额（申请状态=申请人放弃申请）
    id_data['jyzx_sum_Inquiry_Aborted_' + prdname] = prddata[prddata['borrowState']==4][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n内申请总金额（申请状态=审核中）
    id_data['jyzx_sum_Inquiry_Checking_' + prdname] = prddata[prddata['borrowState']==5][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n内申请总金额（申请状态=批贷已放款/待放款）
    id_data['jyzx_sum_Inquiry_Pass_' + prdname] = prddata[prddata['borrowState'].isin([2,3,6])][['certNo','borrowAmount']].groupby('certNo').sum()

    #### n天内申请金额通过率
    id_data['jyzx_sum_Inquiry_Pass_rate_' + prdname] =\
    id_data[['jyzx_sum_Inquiry_' + prdname,'jyzx_sum_Inquiry_Pass_' + prdname]]\
    .apply(lambda x:round(x[1]/x[0],2) if x[0]>0 else np.nan, axis=1)
    #########?????WHY高了两个点
    #print(id_data['jyzx_sum_Inquiry_Pass_rate_' + prdname])
    

    
    
    
    #### n天内申请最大金额    
    id_data['jyzx_max_Inquiry_' + prdname] = prddata[['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内申请最大金额（申请状态=未知）
    id_data['jyzx_max_Inquiry_Unknown_' + prdname] = prddata[prddata['borrowState']==0][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n内申请最大金额（申请状态=拒贷）
    id_data['jyzx_max_Inquiry_Rejected_' + prdname] = prddata[prddata['borrowState']==1][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n内申请最大金额（申请状态=批贷已放款）
    id_data['jyzx_max_Inquiry_Approved_' + prdname] = prddata[prddata['borrowState']==2][['certNo','borrowAmount']].groupby('certNo').max()

    #### n内申请最大金额（申请状态=待放款）
    id_data['jyzx_max_Inquiry_Awaiting_' + prdname] = prddata[prddata['borrowState'].isin([3,6])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n内申请最大金额（申请状态=申请人放弃申请）
    id_data['jyzx_max_Inquiry_Aborted_' + prdname] = prddata[prddata['borrowState']==4][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n内申请最大金额（申请状态=审核中）
    id_data['jyzx_max_Inquiry_Checking_' + prdname] = prddata[prddata['borrowState']==5][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n内申请最大金额（申请状态=批贷已放款/待放款）
    id_data['jyzx_max_Inquiry_Pass_' + prdname] = prddata[prddata['borrowState'].isin([2,3,6])][['certNo','borrowAmount']].groupby('certNo').max()

    
    
    

    #### n天内放款最大期数
    id_data['jyzx_max_Period_' + prdname] = prddata[prddata['borrowState'].isin([2,3,6])][['certNo','loanPeriod']].groupby('certNo').max()
    
    #### n天内放款最小期数
    id_data['jyzx_min_Period_' + prdname] = prddata[prddata['borrowState'].isin([2,3,6])][['certNo','loanPeriod']].groupby('certNo').min()

    #### n天内放款平均期数
    id_data['jyzx_mean_Period_' + prdname] = prddata[prddata['borrowState'].isin([2,3,6])][['certNo','loanPeriod']].groupby('certNo').mean().apply(lambda x:round(x,2))


In [36]:
#### PART 2 放款状态

for prdname, prdtime in period:
    prddata = pd_data[pd_data['contractDate']<prdtime]
    
    #### n天内总放款次数（还款状态=未知）
    id_data['jyzx_n_Repay_Unknown_' + prdname] = prddata[prddata['repayState'].isin([0])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=正常）
    id_data['jyzx_n_Repay_Normal_' + prdname] = prddata[prddata['repayState'].isin([1])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M6+）
    id_data['jyzx_n_Repay_M6+_' + prdname] = prddata[prddata['repayState'].isin([8])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=已还清）
    id_data['jyzx_n_Repay_Clear_' + prdname] = prddata[prddata['repayState'].isin([9])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=正常或已还清）
    id_data['jyzx_n_Repay_Paid_' + prdname] = prddata[prddata['repayState'].isin([1,9])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=带M的值）
    id_data['jyzx_n_Repay_Odue_' + prdname] = prddata[prddata['repayState'].isin([2,3,4,5,6,7,8])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M1）
    id_data['jyzx_n_Repay_M1_' + prdname] = prddata[prddata['repayState'].isin([2])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M1及以上）
    id_data['jyzx_n_Repay_M1plus_' + prdname] = prddata[prddata['repayState'].isin([2,3,4,5,6,7,8])][['certNo','repayState']].groupby('certNo').count()
    ##########（还款状态=带M的值）#############?????????
    
    #### n天内总放款次数（还款状态=M2）
    id_data['jyzx_n_Repay_M2_' + prdname] = prddata[prddata['repayState'].isin([3])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M2及以上）
    id_data['jyzx_n_Repay_M2plus_' + prdname] = prddata[prddata['repayState'].isin([3,4,5,6,7,8])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M3）
    id_data['jyzx_n_Repay_M3_' + prdname] = prddata[prddata['repayState'].isin([4])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M3及以上）
    id_data['jyzx_n_Repay_M3plus_' + prdname] = prddata[prddata['repayState'].isin([4,5,6,7,8])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M4）
    id_data['jyzx_n_Repay_M4_' + prdname] = prddata[prddata['repayState'].isin([5])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M4及以上）
    id_data['jyzx_n_Repay_M4plus_' + prdname] = prddata[prddata['repayState'].isin([5,6,7,8])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M5）
    id_data['jyzx_n_Repay_M5_' + prdname] = prddata[prddata['repayState'].isin([6])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M5及以上）
    id_data['jyzx_n_Repay_M5plus_' + prdname] = prddata[prddata['repayState'].isin([6,7,8])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M6）
    id_data['jyzx_n_Repay_M6_' + prdname] = prddata[prddata['repayState'].isin([7])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=M6及以上）
    id_data['jyzx_n_Repay_M6plus_' + prdname] = prddata[prddata['repayState'].isin([7,8])][['certNo','repayState']].groupby('certNo').count()
    
    #### n天内总放款次数（还款状态=除已还清以外的值）
    id_data['jyzx_n_Repay_Unclear_' + prdname] = prddata[prddata['repayState'].isin([0,1,2,3,4,5,6,7,8])][['certNo','repayState']].groupby('certNo').count()
    
    
    
    
    #### n天内总放款金额（还款状态=未知）
    id_data['jyzx_sum_Repay_Unknown_' + prdname] = prddata[prddata['repayState'].isin([0])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=正常）
    id_data['jyzx_sum_Repay_Normal_' + prdname] = prddata[prddata['repayState'].isin([1])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M6+）
    id_data['jyzx_sum_Repay_M6+_' + prdname] = prddata[prddata['repayState'].isin([8])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=已还清）
    id_data['jyzx_sum_Repay_Clear_' + prdname] = prddata[prddata['repayState'].isin([9])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=正常或已还清）
    id_data['jyzx_sum_Repay_Paid_' + prdname] = prddata[prddata['repayState'].isin([1,9])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=带M的值）
    id_data['jyzx_sum_Repay_Odue_' + prdname] = prddata[prddata['repayState'].isin([2,3,4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M1）
    id_data['jyzx_sum_Repay_M1_' + prdname] = prddata[prddata['repayState'].isin([2])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M1及以上）
    id_data['jyzx_sum_Repay_M1plus_' + prdname] = prddata[prddata['repayState'].isin([2,3,4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').sum()
    ##########（还款状态=带M的值）#############?????????
    
    #### n天内总放款金额（还款状态=M2）
    id_data['jyzx_sum_Repay_M2_' + prdname] = prddata[prddata['repayState'].isin([3])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M2及以上）
    id_data['jyzx_sum_Repay_M2plus_' + prdname] = prddata[prddata['repayState'].isin([3,4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M3）
    id_data['jyzx_sum_Repay_M3_' + prdname] = prddata[prddata['repayState'].isin([4])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M3及以上）
    id_data['jyzx_sum_Repay_M3plus_' + prdname] = prddata[prddata['repayState'].isin([4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M4）
    id_data['jyzx_sum_Repay_M4_' + prdname] = prddata[prddata['repayState'].isin([5])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M4及以上）
    id_data['jyzx_sum_Repay_M4plus_' + prdname] = prddata[prddata['repayState'].isin([5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M5）
    id_data['jyzx_sum_Repay_M5_' + prdname] = prddata[prddata['repayState'].isin([6])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M5及以上）
    id_data['jyzx_sum_Repay_M5plus_' + prdname] = prddata[prddata['repayState'].isin([6,7,8])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M6）
    id_data['jyzx_sum_Repay_M6_' + prdname] = prddata[prddata['repayState'].isin([7])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=M6及以上）
    id_data['jyzx_sum_Repay_M6plus_' + prdname] = prddata[prddata['repayState'].isin([7,8])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总放款金额（还款状态=除已还清以外的值）
    id_data['jyzx_sum_Repay_Unclear_' + prdname] = prddata[prddata['repayState'].isin([0,1,2,3,4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    
    
    
    #### n天内最大放款金额（还款状态=未知）
    id_data['jyzx_max_Repay_Unknown_' + prdname] = prddata[prddata['repayState'].isin([0])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=正常）
    id_data['jyzx_max_Repay_Normal_' + prdname] = prddata[prddata['repayState'].isin([1])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M6+）
    id_data['jyzx_max_Repay_M6+_' + prdname] = prddata[prddata['repayState'].isin([8])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=已还清）
    id_data['jyzx_max_Repay_Clear_' + prdname] = prddata[prddata['repayState'].isin([9])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=正常或已还清）
    id_data['jyzx_max_Repay_Paid_' + prdname] = prddata[prddata['repayState'].isin([1,9])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=带M的值）
    id_data['jyzx_max_Repay_Odue_' + prdname] = prddata[prddata['repayState'].isin([2,3,4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M1）
    id_data['jyzx_max_Repay_M1_' + prdname] = prddata[prddata['repayState'].isin([2])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M1及以上）
    id_data['jyzx_max_Repay_M1plus_' + prdname] = prddata[prddata['repayState'].isin([2,3,4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').max()
    ##########（还款状态=带M的值）#############?????????
    
    #### n天内最大放款金额（还款状态=M2）
    id_data['jyzx_max_Repay_M2_' + prdname] = prddata[prddata['repayState'].isin([3])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M2及以上）
    id_data['jyzx_max_Repay_M2plus_' + prdname] = prddata[prddata['repayState'].isin([3,4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M3）
    id_data['jyzx_max_Repay_M3_' + prdname] = prddata[prddata['repayState'].isin([4])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M3及以上）
    id_data['jyzx_max_Repay_M3plus_' + prdname] = prddata[prddata['repayState'].isin([4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M4）
    id_data['jyzx_max_Repay_M4_' + prdname] = prddata[prddata['repayState'].isin([5])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M4及以上）
    id_data['jyzx_max_Repay_M4plus_' + prdname] = prddata[prddata['repayState'].isin([5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M5）
    id_data['jyzx_max_Repay_M5_' + prdname] = prddata[prddata['repayState'].isin([6])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M5及以上）
    id_data['jyzx_max_Repay_M5plus_' + prdname] = prddata[prddata['repayState'].isin([6,7,8])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M6）
    id_data['jyzx_max_Repay_M6_' + prdname] = prddata[prddata['repayState'].isin([7])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=M6及以上）
    id_data['jyzx_max_Repay_M6plus_' + prdname] = prddata[prddata['repayState'].isin([7,8])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大放款金额（还款状态=除已还清以外的值）
    id_data['jyzx_max_Repay_Unclear_' + prdname] = prddata[prddata['repayState'].isin([0,1,2,3,4,5,6,7,8])][['certNo','borrowAmount']].groupby('certNo').max()


In [37]:
#### PART 3 申请类型

for prdname, prdtime in period:
    prddata = pd_data[pd_data['contractDate']<prdtime]
    
    #### n天内总申请次数（申请类型=未知）
    id_data['jyzx_n_Type_Unknown_' + prdname] = prddata[prddata['borrowType'].isin([0])][['certNo','borrowType']].groupby('certNo').count()

    #### n天内总申请次数（申请类型=个人信贷）
    id_data['jyzx_n_Type_PersonalCredit_' + prdname] = prddata[prddata['borrowType'].isin([1])][['certNo','borrowType']].groupby('certNo').count()
    
    #### n天内总申请次数（申请类型=个人抵押）
    id_data['jyzx_n_Type_PersonalDeposit_' + prdname] = prddata[prddata['borrowType'].isin([2])][['certNo','borrowType']].groupby('certNo').count()
    
    #### n天内总申请次数（申请类型=个人信贷or个人抵押）
    id_data['jyzx_n_Type_Personal_' + prdname] = prddata[prddata['borrowType'].isin([1,2])][['certNo','borrowType']].groupby('certNo').count()
    
    #### n天内总申请次数（申请类型=企业信贷）
    id_data['jyzx_n_Type_CompanyCredit_' + prdname] = prddata[prddata['borrowType'].isin([3])][['certNo','borrowType']].groupby('certNo').count()
    
    #### n天内总申请次数（申请类型=企业抵押）
    id_data['jyzx_n_Type_CompanyDeposit_' + prdname] = prddata[prddata['borrowType'].isin([4])][['certNo','borrowType']].groupby('certNo').count()
    
    #### n天内总申请次数（申请类型=企业信贷or企业抵押）
    id_data['jyzx_n_Type_Company_' + prdname] = prddata[prddata['borrowType'].isin([3,4])][['certNo','borrowType']].groupby('certNo').count()
    
    
    
    
    #### n天内总申请金额（申请类型=未知）
    id_data['jyzx_sum_Type_Unknown_' + prdname] = prddata[prddata['borrowType'].isin([0])][['certNo','borrowAmount']].groupby('certNo').sum()

    #### n天内总申请金额（申请类型=个人信贷）
    id_data['jyzx_sum_Type_PersonalCredit_' + prdname] = prddata[prddata['borrowType'].isin([1])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总申请金额（申请类型=个人抵押）
    id_data['jyzx_sum_Type_PersonalDeposit_' + prdname] = prddata[prddata['borrowType'].isin([2])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总申请金额（申请类型=个人信贷or个人抵押）
    id_data['jyzx_sum_Type_Personal_' + prdname] = prddata[prddata['borrowType'].isin([1,2])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总申请金额（申请类型=企业信贷）
    id_data['jyzx_sum_Type_CompanyCredit_' + prdname] = prddata[prddata['borrowType'].isin([3])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总申请金额（申请类型=企业抵押）
    id_data['jyzx_sum_Type_CompanyDeposit_' + prdname] = prddata[prddata['borrowType'].isin([4])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    #### n天内总申请金额（申请类型=企业信贷or企业抵押）
    id_data['jyzx_sum_Type_Company_' + prdname] = prddata[prddata['borrowType'].isin([3,4])][['certNo','borrowAmount']].groupby('certNo').sum()
    
    
    
    
    #### n天内最大申请金额（申请类型=未知）
    id_data['jyzx_max_Type_Unknown_' + prdname] = prddata[prddata['borrowType'].isin([0])][['certNo','borrowAmount']].groupby('certNo').max()

    #### n天内最大申请金额（申请类型=个人信贷）
    id_data['jyzx_max_Type_PersonalCredit_' + prdname] = prddata[prddata['borrowType'].isin([1])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大申请金额（申请类型=个人抵押）
    id_data['jyzx_max_Type_PersonalDeposit_' + prdname] = prddata[prddata['borrowType'].isin([2])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大申请金额（申请类型=个人信贷or个人抵押）
    id_data['jyzx_max_Type_Personal_' + prdname] = prddata[prddata['borrowType'].isin([1,2])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大申请金额（申请类型=企业信贷）
    id_data['jyzx_max_Type_CompanyCredit_' + prdname] = prddata[prddata['borrowType'].isin([3])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大申请金额（申请类型=企业抵押）
    id_data['jyzx_max_Type_CompanyDeposit_' + prdname] = prddata[prddata['borrowType'].isin([4])][['certNo','borrowAmount']].groupby('certNo').max()
    
    #### n天内最大申请金额（申请类型=企业信贷or企业抵押）
    id_data['jyzx_max_Type_Company_' + prdname] = prddata[prddata['borrowType'].isin([3,4])][['certNo','borrowAmount']].groupby('certNo').max()


In [38]:
#### PART4 逾期相关
for prdname, prdtime in period:
    prddata = pd_data[pd_data['contractDate']<prdtime]
    
    #### n天内总逾期次数
    id_data['jyzx_n_arrears_' + prdname] = prddata[prddata['arrearsAmount']>0][['certNo','arrearsAmount']].groupby('certNo').count()
 
    #### n天内总逾期金额
    id_data['jyzx_sum_arrears_' + prdname] = prddata[prddata['arrearsAmount']>0][['certNo','arrearsAmount']].groupby('certNo').sum()
    
    #### n天内总逾期次数
    id_data['jyzx_max_arrears_' + prdname] = prddata[prddata['arrearsAmount']>0][['certNo','arrearsAmount']].groupby('certNo').max()

In [39]:
#### PART5 时间相关

#### 最晚一次申请距今天数
id_data['jyzx_min_Inquiry_date_gap'] = pd_data[['contractDate','certNo']].groupby('certNo').min().apply(lambda x:x/60/60//24)

#### 最早一次申请距今天数
id_data['jyzx_max_Inquiry_date_gap'] = pd_data[['contractDate','certNo']].groupby('certNo').max().apply(lambda x:x/60/60//24)

#### 最晚一次逾期距今天数
id_data['jyzx_min_Odue_date_gap'] = pd_data[pd_data['arrearsAmount']>0][['contractDate','certNo']].groupby('certNo').min().apply(lambda x:x/60/60//24)

#### 最早一次逾期距今天数
id_data['jyzx_max_Odue_date_gap'] = pd_data[pd_data['arrearsAmount']>0][['contractDate','certNo']].groupby('certNo').max().apply(lambda x:x/60/60//24)


In [40]:
id_data.fillna(0,inplace=True)

# reset the index, the old index is added as a column, and a new sequential index is used
id_data = id_data.reset_index().rename(columns={'index':'certNo'})
#id_data.to_csv('jyzx_loan_character.csv', index=False)
id_data = pd.DataFrame(id_data)
id_data.to_excel('91_loan_1_from_json_character.xlsx',index=False)

In [42]:
id_data

,certNo,jyzx_n_Inquiry_10d,jyzx_n_Inquiry_Unknown_10d,jyzx_n_Inquiry_Rejected_10d,jyzx_n_Inquiry_Approved_10d,jyzx_n_Inquiry_Awaiting_10d,jyzx_n_Inquiry_Aborted_10d,jyzx_n_Inquiry_Checking_10d,jyzx_n_Inquiry_Pass_10d,jyzx_n_Inquiry_Pass_rate_10d,...,jyzx_n_arrears_360d,jyzx_sum_arrears_360d,jyzx_max_arrears_360d,jyzx_n_arrears_all,jyzx_sum_arrears_all,jyzx_max_arrears_all,jyzx_min_Inquiry_date_gap,jyzx_max_Inquiry_date_gap,jyzx_min_Odue_date_gap,jyzx_max_Odue_date_gap
0,110101197404034020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,661.0,0.0,0.0
1,110101198301072010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40.0,17755.0,0.0,0.0
2,110101198607042025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,95.0,17880.0,0.0,0.0
3,110101198707040545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,378.0,409.0,0.0,0.0
4,110101198806040516,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,73.0,0.0,0.0
5,110102197612020818,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,772.0,0.0,0.0
6,110102198107050439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,666.0,666.0,1.0,666.0,666.0,21.0,478.0,163.0,163.0
7,110102198507012327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,86.0,17881.0,0.0,0.0
8,110102199103193031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18.0,1398.0,0.0,0.0
9,110102199103300415,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,571.0,0.0,0.0
